# Clustering Crypto

In [50]:
# Initial imports
import requests
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [2]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [29]:
print(f"Current working directory:{Path.cwd()}")

Current working directory:/Users/aalfiler/Sites/hw_AWS/AWS/Code


In [30]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [31]:
# Alternatively, use the provided csv file:
file_path = Path("/Users/aalfiler/Sites/hw_AWS/AWS/Code/Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [32]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.drop(columns=["Unnamed: 0"])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [33]:
# Keep only cryptocurrencies that are trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [34]:
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144 entries, 0 to 1247
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [35]:
IsTrading_check = crypto_df['IsTrading'].unique()
IsTrading_check

array([ True])

In [ ]:
# Keep only cryptocurrencies with a working algorithm
???

In [36]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns=["IsTrading"])
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [37]:
crypto_df.shape

(1144, 5)

In [38]:
crypto_df.isnull().sum()

CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [39]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna().copy()

In [40]:
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [41]:
crypto_df.shape

(685, 5)

In [46]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df[crypto_df['TotalCoinsMined'] == 0].index, inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [51]:
crypto_df['TotalCoinsMined'].replace('', np.nan, inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [52]:
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [54]:
# Drop rows where there are 'N/A' text values
to_drop = ['N/A']
crypto_df.isin(to_drop)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,False,False,False,False,False
2,False,False,False,False,False
5,False,False,False,False,False
7,False,False,False,False,False
8,False,False,False,False,False
...,...,...,...,...,...
1238,False,False,False,False,False
1242,False,False,False,False,False
1245,False,False,False,False,False
1246,False,False,False,False,False


In [57]:
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [58]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName_df = crypto_df['CoinName']
CoinName_df.head()

0      42 Coin
2      404Coin
5    EliteCoin
7      Bitcoin
8     Ethereum
Name: CoinName, dtype: object

In [59]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df2 = crypto_df.drop(columns="CoinName")
crypto_df2.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [60]:
# Create dummy variables for text features
crypto_df2_encoded = pd.get_dummies(crypto_df2, columns=["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply"])
crypto_df2_encoded.head()

,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,Algorithm_Counterparty,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
crypto_df2_path = Path("/Users/aalfiler/Sites/hw_AWS/AWS/Code/Resources/crypto_data_encoded.csv")
crypto_df2_encoded.to_csv(file_path, index=False)

In [64]:
# Standardize data
data_scaler = StandardScaler()

In [65]:
data_scaler.fit(crypto_df2_encoded)

StandardScaler()

In [67]:
crypto_df2_scaled = data_scaler.transform(crypto_df2_encoded)
crypto_df2_scaled[:5]

array([[-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555]])

### Reducing Dimensions Using PCA

In [70]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_df2_pca = pca.fit_transform(crypto_df2_scaled)

In [71]:
# Create a DataFrame with the principal components data
crypto_df2_pca = pd.DataFrame(
    data=crypto_df2_pca, columns=["principal component 1", "principal component 2", 'principal component 3']
)
crypto_df2_pca.head()

,principal component 1,principal component 2,principal component 3
0,-0.084723,0.225830,0.136834
1,-0.028226,0.288352,0.107690
2,-0.139875,-0.876731,0.496848
3,0.053066,-0.001264,-0.436843
4,-0.068184,0.062031,0.165760


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [76]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df2_pca)
    inertia.append(km.inertia_)

In [77]:
# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [82]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(crypto_df2_pca)
# Predict clusters
predictions = model.predict(crypto_df2_pca)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
crypto_df2_pca["class"] = model.labels_
crypto_df2_pca.head()

,principal component 1,principal component 2,principal component 3,class
0,-0.084723,0.225830,0.136834,0
1,-0.028226,0.288352,0.107690,0
2,-0.139875,-0.876731,0.496848,0
3,0.053066,-0.001264,-0.436843,0
4,-0.068184,0.062031,0.165760,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [83]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crypto_df2_pca,
    x="principal component 3",
    y="principal component 2",
    z="principal component 1",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
